# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [3]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local']


In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our new database
pprint(db.list_collection_names())

[]


In [8]:
# review a document in the establishments collection
establishment = db['establishments'].find_one()
pprint(establishment)

None


In [22]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [23]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "name": "Penang Flavours",
    "address": "123 Main Street",
    "city": "Greenwich",
    "postcode": "SE10 8JA",
    "rating_value": None,
    "hygiene_score": None,
    "cuisine": ["Halal"],
    "last_inspection_date": None
}

In [24]:
# Insert the new restaurant into the collection
insert_result = db.establishments.insert_one(new_restaurant_data)

In [25]:
# Check that the new restaurant was inserted
print("Inserted document ID:", insert_result.inserted_id)

Inserted document ID: 646e9f8b50daa858f5e6f314


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [42]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = db.establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})

  

3. Update the new restaurant with the `BusinessTypeID` you found.

In [38]:
# Update the new restaurant with the correct BusinessTypeID
if business_type is not None: 
    update_result = db.establishments.update_one(
        {"name": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
    )

In [41]:
# Confirm that the new restaurant was updated
    print(update_result)
else:
    print("Business type not found. Unable to update the new restaurant.")

IndentationError: unexpected indent (4192150822.py, line 2)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(dover_count)

0


In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
deleted_count = delete_result.deleted_count
print("Number of establishments deleted:", deleted_count)

Number of establishments deleted: 0


In [28]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments remaining with LocalAuthorityName as 'Dover':", remaining_count)

Number of establishments remaining with LocalAuthorityName as 'Dover': 0


In [29]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()
pprint(remaining_document)


{'_id': ObjectId('646e9e8750daa858f5e6f313'),
 'address': '123 Main Street',
 'city': 'Greenwich',
 'cuisine': ['Halal'],
 'hygiene_score': None,
 'last_inspection_date': None,
 'name': 'Penang Flavours',
 'postcode': 'SE10 8JA',
 'rating_value': None}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many({}, [
    {'$set': {'latitude': {'$toDouble': '$latitude'}}},
    {'$set': {'longitude': {'$toDouble': '$longitude'}}}
])


Use `update_many` to convert `RatingValue` to integer numbers.

In [31]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [32]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many({}, [
    {'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}
])

In [33]:
# Check that the coordinates and rating value are now numbers
example_document = db.establishments.find_one()
pprint(example_document)

{'RatingValue': None,
 '_id': ObjectId('646e9e8750daa858f5e6f313'),
 'address': '123 Main Street',
 'city': 'Greenwich',
 'cuisine': ['Halal'],
 'hygiene_score': None,
 'last_inspection_date': None,
 'latitude': None,
 'longitude': None,
 'name': 'Penang Flavours',
 'postcode': 'SE10 8JA',
 'rating_value': None}
